<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       VantageCloud Lake Demos Overview and Administration
  <br>
       <img id="teradata-logo" src="images/TeradataLogo.png" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>
<hr>

<br>

<b style = 'font-size:24px;font-family:Arial;color:#00233C'>Demo Administration Notebook</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>After running the Environment setup, the system will have a set of users, databases, tables, and Compute Group/Profiles/Roles.  This notebook will serve as an overview of the additional demos that can be run from this environment, as well as some SQL for administering Clusters and other resources.</p>

<b style = 'font-size:20px;font-family:Arial;color:#00233C'>Demo Checklist</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>To initiate the configuration of the environment used for these demos, perform the steps in "Environment Setup Automation" below; either by running the Jupyter Notebook or the Airflow DAG.  Prior to running these notebooks, complete the following:</p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'><li>Edit vars.json to reflect the target environment</li>
    <li>Validate other environment and hierarchy settings in vars.json</li>
    <li>Clusters are set up to be active during nominal business hours <b>USA TIME</b>.  Adjust as necessary in the notebook or DAG</li>
    <li>If using Airflow:
        <ul><li>Upload the new vars.json to Variables in Airflow</li>
            <li>Airflow script does not load any data, execute the "Section 3 - Load Data" cells in the Demo_Environment_Setup_Automation notebook.</li>
        </ul></li>
    <li>When the setup is complete, use this notebook to check cluster status, suspend/resume as needed</li>
    </ol>

<hr>


<hr>

<b style = 'font-size:24px;font-family:Arial;color:#00233C'>Admin Notebook SQL</b>

<hr>

<b style = 'font-size:18px;font-family:Arial;color:#00233C'>Step 1 - Create or replace a connection to the Lake Environment </b>

<b style = 'font-size:16px;font-family:Arial;color:#00233C'>Using line magics (%):</b>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>List available connections</li>
    <li>Create the new one, or change the existing one to the correct host/IP</li>
    <li>Connect as cgadmin (password as set during environment setup)</li>

In [ ]:
%lsconnect

In [3]:
%addconnect name=lake_demo, host=XXX.XXX.XXX.XXX

Success: 'lake_demo' connection added

In [14]:
%chconnect name=lake_demo, host=XXX.XXX.XXX.XXX

Success: 'lake_demo' connection modified. Disconnect and re-connect for the modified values to take effect. 


In [42]:
%disconnect lake_demo

Success: 'lake_demo' disconnected

In [43]:
%connect lake_demo, user=cgadmin

Password: ········


Success: 'lake_demo' connection established and activated for user 'cgadmin'


<b style = 'font-size:18px;font-family:Arial;color:#00233C'>Cluster Status and Administration</b>
<hr>

<b style = 'font-size:16px;font-family:Arial;color:#00233C'>Clusters Status</b>

In [47]:
SELECT * FROM DBC.ComputeStatusV

,ComputeProfileName,InstanceName,ComputeGroupName,ComputeMapName,ComputeInstanceType,CurrentState,LastReqState,LastStartTime,LastEndTime
1,CP_BUSGRPA_ANL_1_SML,0deleq,CG_BUSGRPA_ANL,TD_COMPUTE_SMALL,ANALYTIC,Active,SUSPEND,2023-02-13 15:16:28 +0000 UTC,
2,CP_BUSGRPA_STD_1_SML,xr97u1,CG_BUSGRPA_STD,TD_COMPUTE_SMALL,STANDARD,Active,SUSPEND,2023-02-13 15:26:25 +0000 UTC,
3,CP_BUSGRPA_STD_1_SML,96921l,CG_BUSGRPA_STD,TD_COMPUTE_SMALL,STANDARD,Terminated,SUSPEND,2023-02-13 15:16:29 +0000 UTC,


<hr>

<b style = 'font-size:16px;font-family:Arial;color:#00233C'>Suspend/Resume</b>

In [45]:
SUSPEND COMPUTE FOR COMPUTE PROFILE CP_BUSGRPA_ANL_1_SML IN CG_BUSGRPA_ANL;

Success: 1 rows affected

In [46]:
SUSPEND COMPUTE FOR COMPUTE PROFILE CP_BUSGRPA_STD_1_SML IN CG_BUSGRPA_STD;

Success: 1 rows affected

In [48]:
RESUME COMPUTE FOR COMPUTE PROFILE CP_BUSGRPA_ANL_1_SML IN CG_BUSGRPA_ANL;

Success: 1 rows affected

In [92]:
RESUME COMPUTE FOR COMPUTE PROFILE CP_BUSGRPA_STD_1_SML IN CG_BUSGRPA_STD;

Success: 1 rows affected

<hr>

<b style = 'font-size:16px;font-family:Arial;color:#00233C'>DROP Clusters and Profiles</b>

In [59]:
DROP COMPUTE PROFILE CP_BUSGRPA_ANL_1_SML in COMPUTE GROUP CG_BUSGRPA_ANL;

Success: 1 rows affected

In [4]:
DROP COMPUTE PROFILE CP_BUSGRPA_STD_1_SML in COMPUTE GROUP CG_BUSGRPA_STD;

Success: 1 rows affected

In [61]:
DROP COMPUTE GROUP CG_BUSGRPA_ANL;

Success: 16 rows affected

In [61]:
DROP COMPUTE GROUP CG_BUSGRPA_STD;

Success: 16 rows affected

<hr>

<b style = 'font-size:18px;font-family:Arial;color:#00233C'>SYSDBA Login</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>For the case of needing to do something not granted in the setup</p>

In [49]:
%disconnect lake_demo

Success: 'lake_demo' disconnected

In [1]:
%connect lake_demo, user=sysdba

Password: ········


Success: 'lake_demo' connection established and activated for user 'sysdba'


In [52]:
SELECT * FROM DBC.DBCInfo

,InfoKey,InfoData
1,VERSION,20.00.03.28
2,LANGUAGE SUPPORT MODE,Standard
3,RELEASE,20.00.03.28


In [2]:
select upper(TableName) as analytical_function_td, databasename, tablekind, version, requesttext 
from dbc.tables 
where tablekind = 'L' order by 1

,analytical_function_td,DatabaseName,TableKind,Version,RequestText
1,AGGGEOM,TD_SYSFNLIB,L,1,
2,ANTISELECT,TD_SYSFNLIB,L,1,
3,APPLY,TD_SYSFNLIB,L,1,
4,ATTRIBUTION,TD_SYSFNLIB,L,1,
5,AVROCONTAINERSPLIT,TD_SYSFNLIB,L,1,
6,CALCMATRIX,TD_SYSFNLIB,L,1,
7,CHISQUAREMAP,TD_SYSFNLIB,L,1,
8,CHISQUAREREDUCE,TD_SYSFNLIB,L,1,
9,COLSUMMAP,TD_SYSFNLIB,L,1,
10,COLSUMREDUCE,TD_SYSFNLIB,L,1,
